In [14]:

# Libraries
import os
import sys
import json
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150
import numpy as np
import pyro
import pyro.distributions as dist
from pyro.infer.mcmc.api import MCMC
from pyro.infer.mcmc import NUTS, HMC
import torch
import torch.distributions.constraints as constraints
from torch.nn.functional import pad
from tqdm import trange
from sklearn.datasets import make_moons

# For ADVI
from pyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO
from pyro.contrib.autoguide import AutoDiagonalNormal
from pyro.optim import Adam

# For GP
import pyro.contrib.gp as gp

# Default to double precision for torch objects.
torch.set_default_dtype(torch.float64)

In [15]:
def gen_grid(X, n, return_each=False, eps=0):
    x0, x1 = np.meshgrid(np.linspace(X[:, 0].min()-eps, X[:, 0].max()+eps, n),
                         np.linspace(X[:, 1].min()-eps, X[:, 1].max()+eps, n))
    
    Y = np.stack([x0.ravel(), x1.ravel()], axis=-1)
    
    if return_each:
        return Y, x0, x1
    else:
        return Y

def predict(samples, i, X, Xnew, eps=1e-6):
    kernel = gp.kernels.RBF(2, samples['alpha'][i], samples['rho'][i])
    Nnew = Xnew.shape[0]
    
    f = compute_f(samples['alpha'][i],
                  samples['rho'][i],
                  samples['beta'][i],
                  samples['eta'][i], X)
    
    with torch.no_grad():
        gpr = gp.models.GPRegression(X, f, kernel)
        mean, cov = gpr(Xnew, full_cov=True)
        
    fhat = dist.MultivariateNormal(
        mean, cov + torch.eye(Nnew) * eps
    ).sample()
    
    return fhat.sigmoid().numpy()

def plot_data(X, y, **kwargs):
    colors = np.array(['blue', 'red'])
    plt.scatter(X[:, 0], X[:, 1], c=colors[y], **kwargs)

def plot_kernel_params(samples, algo, kernel_params=['alpha', 'rho', 'beta'],
                       figsize=(8, 3)):
    plt.figure(figsize=figsize * 1.2)
    for i in range(len(kernel_params)):
        plt.subplot(1, len(kernel_params), i + 1)
        param = kernel_params[i]
        plt.hist(samples[param], bins=30, density=True)
        plt.xlabel(param)
        plt.ylabel('density')
        plt.title(f"Histograhm of {param} ({algo})")
    plt.tight_layout()   

def plot_uq(samples, X, Xnew, algo, figsize=np.array([8, 3])):
    preds = np.stack([predict(samples, i, X, Xnew)
                    for i in trange(samples['rho'].shape[0])])
    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1)
    plt.contourf(x0, x1, preds.mean(0).reshape(x0.shape), 101,
                cmap=plt.get_cmap('bwr'), vmin=0, vmax=1)
    plt.title(f'Posterior Mean function ({algo})')
    plt.xticks([]); plt.yticks([])
    plt.colorbar()
    plot_data(X, y, edgecolor="orange")

    plt.subplot(1, 2, 2)
    plt.contourf(x0, x1, preds.std(0).reshape(x0.shape), 101,
                cmap=plt.get_cmap('Oranges'), vmin=0)
    plt.title(f'Posterior SD function ({algo})')
    plt.xticks([]); plt.yticks([])
    plt.colorbar()

    plot_data(X, y, edgecolor="black")

    plot_kernel_params(samples, algo, figsize=figsize)

In [19]:
# Make data
# X, y = make_moons(n_samples=50, shuffle=True, noise=0.1, random_state=1)
from func import get_data

# Read data.
X_train,y_train,X_test,y_test,age = get_data(True)

ind_train = age[0].astype("int")
ind_test = age[1].astype("int")

# Prepare data for Pyro model
n_cat = int(y_train.max())
#print('n_cat', n_cat)
n_ind = ind_train.max()
#print('n_ind', n_ind)
X_train_tensor = torch.tensor(X_train.astype('float')).float()
y_train_tensor = torch.tensor(y_train).float()
#print(y_train_tensor)
ind_train = torch.tensor(ind_train) # these are indices, therefore they need to be (long) integers


# Make prediction grid.
Xnew, x0, x1 = gen_grid(X_train_tensor, 30, return_each=True, eps=0.5)
Xnew = torch.from_numpy(Xnew)


['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']


In [23]:
def sq_exp_kernel(d, alpha, rho):
    return alpha * alpha * torch.exp(-0.5 * torch.pow(d / rho, 2))

def compute_f(alpha, rho, beta, eta, X):
    N = X.shape[0]
    D = torch.cdist(X, X)
    K = sq_exp_kernel(D, alpha, rho) + torch.eye(N) * 1e-6
    L = K.cholesky()
    return L.matmul(eta) + beta

# GP Binary Classifier.
def gpc(X, y):
    N = y.shape[0]
    
    # Priors.
    alpha = pyro.sample('alpha', dist.LogNormal(0, 1))
    rho = pyro.sample('rho', dist.LogNormal(0, 1))
    beta = pyro.sample('beta', dist.Normal(0, 1))

    with pyro.plate('latent_response', N):
        eta = pyro.sample('eta', dist.Normal(0, 1))

    # Latent function.
    f = compute_f(alpha, rho, beta, eta, X)
   
    with pyro.plate('response', N):
        pyro.sample('obs', dist.Bernoulli(logits=f), obs=y)

def gpc_modified(*args, **kwargs):
    # Run the original model
    result = gpc(*args, **kwargs)
    
    # Detach any tensors in the result
    if isinstance(result, torch.Tensor):
        result = result.detach()
    
    return result

In [27]:

%%time

### HMC ###
pyro.clear_param_store()

# Set random seed for reproducibility.
pyro.set_rng_seed(2)

# Set up HMC sampler.
# kernel = HMC(gpc, step_size=0.05, trajectory_length=1, adapt_step_size=False, adapt_mass_matrix=False, jit_compile=True)
kernel = HMC(gpc_modified, step_size=0.05, trajectory_length=1, adapt_step_size=False, adapt_mass_matrix=False, jit_compile=True)
hmc = MCMC(kernel, num_samples=500, warmup_steps=500)
hmc.run(X_train_tensor, y_train_tensor.double())

# Get posterior samples
hmc_posterior_samples = hmc.get_samples()

Warmup:   0%|          | 0/1000 [00:00, ?it/s]c:\Users\emmab\anaconda3\lib\site-packages\pyro\poutine\subsample_messenger.py:70: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor(0.0, device=self.device)
Warmup:   1%|▏         | 13/1000 [00:20,  1.64s/it, step size=5.00e-02, acc. prob=0.564]

RuntimeError: you can only change requires_grad flags of leaf variables. If you want to use a computed variable in a subgraph that doesn't require differentiation use var_no_grad = var.detach().